<a href="https://colab.research.google.com/github/sanjivch/100-days-of-gpu/blob/main/CUDA__Day02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version
!nvcc --version
!pip install nvcc4jupyter

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpys6_d6k4".


In [9]:
%%cuda
#include <iostream>
__global__ void matAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if ((i >= N) && (j >= N)) { // guard block
        return ;
    }

    C[i*N+j] = A[i*N+j] + B[i*N+j];

    }

int main() {
    const int N = 5;
    float *A, *B, *C;

    // initialize the input matrices
    A = (float *)malloc( N*N* sizeof(float));
    B = (float *)malloc(N*N* sizeof(float));
    C = (float *)malloc(N*N * sizeof(float));

    // Hardcode values for A, B and C
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i * N + j] = 10.0f;
            B[i * N + j] = 20.0f;
            C[i * N + j] = 0.0f;
        }
    }

    float *d_a, *d_b,*d_c;

    // Allocate memory on host
    cudaMalloc((void **)&d_a, N*N*sizeof(float));
    cudaMalloc((void **)&d_b, N*N*sizeof(float));
    cudaMalloc((void **)&d_c, N*N*sizeof(float));

    // Transfer to device
    cudaMemcpy(d_a, A, N*N*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, B, N*N*sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 dimBlock(32, 16);
    dim3 dimGrid(ceil(N / 32.0f), ceil(N/ 16.0f));
    matAdd<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Synchronize threads on device
    cudaDeviceSynchronize();

    // Transfer back to host
    cudaMemcpy(C, d_c, N*N*sizeof(float), cudaMemcpyDeviceToHost);

    // Check outputs

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {

            printf("%.4f ",C[i * N + j]);
            printf("%.4f ", A[i * N + j]);
            printf("%.4f ", B[i * N + j]);

        }
            printf("\n");
    }


    // Free resources
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

}

30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 
30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 
30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 
30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 
30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 30.0000 10.0000 20.0000 

